<a href="https://colab.research.google.com/gist/alexiusndoro/f80a03097bd142dbc5672e02aef12760/implied_volatility_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:

import csv
import pandas as pd
import numpy as np
import scipy.stats as si
from math import log,sqrt,exp,pi


In [0]:
class data_reader():
  def __init__(self,file_name):
    self.file_name = str(file_name)
    
  def convert_to_pandas_dataframe(self):
    with open(self.file_name,'r') as csv_file:
      csv_reader = csv.reader(csv_file)
      df= pd.DataFrame(csv_reader)
    return df

      




In [0]:
class Trade():
  def __init__(self,trade_id,underlying_type,underlying,r,T,K,option_type,model_type,market_price):
        self.trade_id = trade_id
        self.underlying_type = underlying_type
        self.underlying = underlying 
        self.K=K
        self.r = r
        self.T= T/365
        self.option_type = option_type
        self.model_type=model_type
        self.market_price = market_price

In [0]:
dr = data_reader("input.csv")
data = dr.convert_to_pandas_dataframe()
list_of_trades = []  
print(data[4:5])


  #each_trade = Trade(line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8])
  #list_of_trades.append(each_trade)



FileNotFoundError: ignored

In [0]:
class Model():
  
  
 

  @staticmethod
  def bs_put(Trade,sigma):
      S, K, r, T = Trade.underlying, Trade.K, Trade.r,Trade.T

      d1 = (log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
      d2 = d1 - sigma * sqrt(T)

      bs_put_price = K * exp(-r * T) * si.norm.cdf(-d2) -S * si.norm.cdf(-d1)  
      return bs_put_price

  
  @staticmethod
  def bs_call(Trade,sigma):
      S, K, r, T = Trade.underlying, Trade.K, Trade.r, Trade.T

  #C is the market price of the option
  #must calculate theoretical BS price using guess sigma 

      d1 = (log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
      d2 = d1 - sigma * sqrt(T)

      bs_call_price = S * si.norm.cdf(d1) - K * exp(-r * T) * si.norm.cdf(d2)

      return bs_call_price

  @staticmethod
  def ba_call(Trade,sigma):
      S, K, r, T = Trade.underlying, Trade.K, Trade.r,Trade.T
      d1  = (S-K)/(sigma*sqrt(T))

      ba_call_price =   ((S - K)*si.norm.cdf(d1) + sigma*sqrt(T)*si.norm.pdf(d1))*exp(-r*T)
      return ba_call_price
 
  @staticmethod
  def ba_put(Trade,sigma):
      S, K, r,T = Trade.underlying, Trade.K, Trade.r, Trade.T
      d1  = (S-K)/(sigma*sqrt(T))

      ba_put_price = ((K-S)*si.norm.cdf(-d1) + sigma*sqrt(T)*si.norm.pdf(d1)) * exp(-r*T)
      return ba_put_price





  
    

In [0]:
trade = Trade(1,"future",105,0.01,30,100,"call","blackscholes",0.25)
trade2 = Trade(1,"future",1.2286,-0.0048,194.1459,1.3582,"call","BlackScholes",0.25)
Model.bs_put(trade2,0.8660054320972261)

In [0]:
class newton_solver: 
   
  @staticmethod
  def calculate_volatility(Trade,vol):
      S, K, r,T, C = Trade.underlying, Trade.K, Trade.r,  Trade.T, Trade.market_price
     

      if Trade.model_type== "BlackScholes": 
        d1 = (log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * sqrt(T))
      elif Trade.model_type== "Bachelier": 
         d1  = (S-K)/(vol*sqrt(T))

      #tolerances
      tolerance = 1e-8
      epsilon =1

      count =0
      max_iteration = 1000
      
      #initial guess for volatility
      #vol = 0.25

      while epsilon > tolerance:
        count+=1
        if count>= max_iteration:
          print('breaking on count')
          return  

        original_vol = vol

        function_value=0   
           #funtion we are finding the root of
        if Trade.model_type == "BlackScholes":   
          if Trade.option_type == "Call":
            function_value = Model.bs_call(Trade,vol) - C
          if Trade.option_type == "Put":
            function_value = Model.bs_put(Trade,vol) - C
        if Trade.model_type == "Bachelier":
          if Trade.option_type == "Call":
             function_value = Model.ba_call(Trade,vol) - C
          if Trade.option_type == "Put":
            function_value = Model.ba_put(Trade,vol) - C  

        
        print(Model.bs_put(Trade,vol))
        vega = S*si.norm.pdf(d1)*sqrt(T)

        vol = -function_value/vega +vol  

        epsilon = abs((vol-original_vol)/original_vol)

        vega = (1 / sqrt(2 * pi)) * S * sqrt(T) * exp(-(si.norm.cdf(d1) ** 2) * 0.5)
      print("number of iterations is", count)
      return vol

In [0]:
trade3 = Trade(1,"future",1.2286,-0.0048,194.1459,1.3582,"Put","BlackScholes",0.391629)
newton_solver.calculate_volatility(trade3,0.25)

In [0]:
data1 = (data[4:5])
trade4 = Trade(data1[0],data1[1],data1[2],data1[3],data1[4],data1[5],data1[6],data1[7],data1[8])
